In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import pandas as pd

train_data = pd.read_csv("Train_Data.csv")
test_data = pd.read_csv("Test_Data.csv")
# Checking the column names
print(train_data.columns)

# Calculate total cost
train_data["total cost"] = (train_data["unit price of residence space"] * train_data["residence space"] + train_data["unit price of building space"] * train_data["building space"]) * train_data["exchange rate"]

# Round the total cost to the nearest integer
train_data["total cost"] = train_data["total cost"].round()

# Dropping the columns "Unit price of residence space" and "Unit price of building space"
train_data = train_data.drop(["unit price of residence space", "unit price of building space"], axis=1)
import numpy as np

# Define the price ranges based on the total cost column
price_ranges = [(0, 300000), (300000, 500000), (500000, 700000), (700000, float('inf'))]

# Define a function to assign each total cost to a price range
def assign_price_range(total_cost):
    for i in range(len(price_ranges)):
        if total_cost >= price_ranges[i][0] and total_cost < price_ranges[i][1]:
            return i + 1

# Create a new column in the train dataset to store the price ranges
train_data['price range'] = train_data['total cost'].apply(assign_price_range)
from sklearn.preprocessing import LabelEncoder

# Combine the training and test datasets
combined_data = pd.concat([train_data, test_data])

# Fit the label encoder on the combined dataset
encoder = LabelEncoder()
encoder.fit(combined_data['district'])

# Apply the label encoder to the district column of both datasets
train_data['district'] = encoder.transform(train_data['district'])
test_data['district'] = encoder.transform(test_data['district'])

# Repeat the process for the city column
encoder = LabelEncoder()
encoder.fit(combined_data['city'])

train_data['city'] = encoder.transform(train_data['city'])
test_data['city'] = encoder.transform(test_data['city'])

# Repeat the process for the zip code column
encoder = LabelEncoder()
encoder.fit(combined_data['zip code'])

train_data['zip code'] = encoder.transform(train_data['zip code'])
test_data['zip code'] = encoder.transform(test_data['zip code'])
from sklearn.model_selection import train_test_split

# Split the train dataset into train and validation datasets
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)
# Define the features and target
features = ['number of rooms', 'security level of the community', 'residence space', 'building space', 'noise level', 'waterfront', 'view', 'air quality level', 'aboveground space ', 'basement space', 'building year', 'decoration year', 'district', 'city', 'zip code']
target = 'price range'
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Split the train_data into four parts for the clients
client_data = np.array_split(train_data, 4)

# Define the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, max_depth=20, random_state=42)

# Define a function to train the model on each client's data
def train_client_model(client_data):
    X = client_data[features]
    y = client_data[target]
    rf_model.fit(X, y)

# Train the model on each client's data
for client in client_data:
    train_client_model(client)

# Define a function to aggregate the model weights from each client
def aggregate_model_weights(client_models):
    avg_model_weights = []
    for weights_list in zip(*client_models):
        avg_weights = np.mean([tree.feature_importances_ for tree in weights_list], axis=0)
        avg_model_weights.append(avg_weights)
    return avg_model_weights

# Define a function to evaluate the model on the validation data
def evaluate_model(model, val_data):
    X_val = val_data[features]
    y_val = val_data[target]
    y_pred = model.predict(X_val)
    return accuracy_score(y_val, y_pred)

# Define the initial global model as the trained model on the first client's data
global_model = rf_model

# Define the number of federated learning rounds
num_rounds = 10

# Perform federated learning
for i in range(num_rounds):
    print("Round:", i+1)
    client_models = []
    for client in client_data:
        client_model = RandomForestClassifier(n_estimators=100, max_depth=20, random_state=42)
        X = client[features]
        y = client[target]
        client_model.fit(X, y)
        client_models.append(client_model)
    avg_model_weights = aggregate_model_weights([model.estimators_ for model in client_models])
    global_model.estimators_ = avg_model_weights
    accuracy = evaluate_model(global_model, val_data)